In [1]:
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv

load_dotenv()

# Initialize the language model
llm = ChatOpenAI(model="qwen-max-latest", temperature=0)

# 设置模型输出约束

In [ ]:
from langchain_core.prompts import PromptTemplate


constrained_prompt = PromptTemplate(
    input_variables=["product", "target_audience", "tone", "word_limit"],
    template="""为 {product} 创建产品描述，目标受众为 {target_audience}。
    使用 {tone} 语气，字数控制在 {word_limit} 以内。
    描述应包含：
    1. 醒目的标题
    2. 三个主要功能
    3. 行动号召

    产品描述：
    """,
)


In [ ]:
from IPython.display import display, Markdown


# Generate the constrained output
input_variables = {
    "product": "智能水瓶",
    "target_audience": "注重健康的千禧一代",
    "tone": "随意而友好",
    "word_limit": "75",
}

chain = constrained_prompt | llm
output = chain.invoke(input_variables).content
display(Markdown(output))

**喝出健康，潮酷随行！**  
智能水瓶来啦！💧 定时提醒喝水、记录每日摄入量、智能温控不怕烫嘴。让健康变简单，喝水也变有趣！✨ 快来入手，和我们一起开启水润生活吧！

# 实现基于规则的生成

In [5]:
job_posting_prompt = PromptTemplate(
    input_variables=["job_title", "company", "location", "experience"],
    template="""在 {location} 的 {company} 创建一条 {job_title} 职位的招聘信息。
    应聘者应具备 {experience} 年工作经验。
    请遵循以下规则：
    1. 以简短的公司描述开头（2 句话）
    2. 列出 5 项关键职责，每项都以一个动词开头
    3. 列出 5 项必备资质，每项都用一句话概括
    4. 以标准化的平等机会声明结尾
    
    输出格式如下：
    公司：[公司描述]
    
    职责：
    - [职责 1]
    - [职责 2]
    - [职责 3]
    - [职责 4]
    - [职责 5]
    
    资质：
    - [资质 1]
    - [资质 2]
    - [资质 3]
    - [资质 4]
    - [资质 5]
    
    平等就业机会：[平等机会声明]
    """,
)

In [6]:
# Generate the rule-based output
input_variables = {
    "job_title": "高级软件工程师",
    "company": "PyTorch研习社",
    "location": "安徽，合肥",
    "experience": "5+",
}

chain = job_posting_prompt | llm
output = chain.invoke(input_variables).content
display(Markdown(output))

公司：PyTorch研习社位于安徽合肥，专注于深度学习和人工智能技术的研究与应用。我们致力于打造创新的解决方案，推动AI技术在各行业的落地与发展。

职责：
- 设计并实现高效的深度学习模型和算法
- 优化现有软件架构以提升系统性能和可扩展性
- 领导团队完成复杂项目，确保按时交付高质量成果
- 分析用户需求并转化为技术解决方案
- 指导初级工程师，分享专业知识以提升团队整体能力

资质：
- 拥有计算机科学或相关领域的本科及以上学历
- 具备5年以上软件开发经验，且有2年以上深度学习框架（如PyTorch）使用经验
- 精通Python/C++编程语言，熟悉常用算法和数据结构
- 对机器学习和深度学习理论有深刻理解，能够解决实际问题
- 良好的沟通能力和团队协作精神，能够在多学科团队中高效工作

平等就业机会：PyTorch研习社是一个提供平等就业机会的雇主，我们欢迎来自不同背景的候选人申请，所有申请将根据个人能力和岗位匹配度进行评估。

# 使用正则表达式解析器进行结构化输出

In [18]:
from langchain.output_parsers.regex import RegexParser


regex_parser = RegexParser(
    regex=r"公司：\s*([\s\S]*?)\n\s*职责：\s*([\s\S]*?)\n\s*资质：\s*([\s\S]*?)\n\s*平等就业机会：\s*([\s\S]*)",
    output_keys=[
        "公司简介",
        "职责",
        "资质",
        "EEO 声明",
    ],
)

parsed_job_posting_prompt = PromptTemplate(
    input_variables=["job_title", "company", "location", "experience"],
    template="""在 {location} 的 {company} 创建一条 {job_title} 职位的招聘信息。
    应聘者应具备 {experience} 年工作经验。
    请遵循以下规则：
    1. 以简短的公司描述开头（2 句话）
    2. 列出 5 项关键职责，每项都以一个动词开头
    3. 列出 5 项必备资质，每项都用一句话概括
    4. 以标准化的平等机会声明结尾
    
    输出格式如下：
    公司：[公司描述]
    
    职责：
    - [职责 1]
    - [职责 2]
    - [职责 3]
    - [职责 4]
    - [职责 5]
    
    资质：
    - [资质 1]
    - [资质 2]
    - [资质 3]
    - [资质 4]
    - [资质 5]
    
    平等就业机会：[平等机会声明]
    """,
)

In [19]:
import re


def clean_output(output):
    for key, value in output.items():
        if isinstance(value, str):
            # Remove leading/trailing whitespace and normalize newlines
            output[key] = re.sub(r"\n\s*", "\n", value.strip())
    return output

In [20]:
# Generate the parsed output
chain = parsed_job_posting_prompt | llm
raw_output = chain.invoke(input_variables).content

# Parse and clean the output
parsed_output = regex_parser.parse(raw_output)
cleaned_output = clean_output(parsed_output)

# Display the parsed output
print("解析后的输出:")
for key, value in cleaned_output.items():
    print(f"{key.upper()}:")
    print(value)
    print()

解析后的输出:
公司简介:
PyTorch研习社是一家专注于深度学习和人工智能技术研究与应用的创新机构，致力于推动PyTorch框架在各行业的落地与发展。

职责:
- 开发高性能的深度学习模型和算法以解决实际问题
- 优化现有代码库的性能并提升其可扩展性
- 指导初级工程师进行技术实践和代码审查
- 与跨职能团队合作设计端到端的AI解决方案
- 跟踪行业最新技术趋势并将其应用于项目中

资质:
- 拥有计算机科学或相关领域的本科及以上学历
- 具备5年以上软件开发经验，且有2年以上深度学习相关工作经验
- 精通Python编程语言及PyTorch框架的使用
- 熟悉分布式系统和云计算平台（如AWS、Azure等）
- 具备良好的沟通能力和团队协作精神

EEO 声明:
PyTorch研习社是一个平等机会雇主，我们欢迎来自不同背景的候选人申请，并致力于打造多元化和包容性的工作环境。



# 实施附加约束

In [21]:
review_prompt = PromptTemplate(
    input_variables=["product", "rating", "pros", "cons", "word_limit"],
    template="""为 {product} 撰写产品评价，并遵循以下限制：
    1. 评价应为 {rating} 星（满分 5 星）
    2. 包含 {pros} 个优点和 {cons} 个缺点
    3. 每个优点和缺点使用 2 到 3 句话
    4. 整篇评价的字数应少于 {word_limit} 个
    5. 以一句话作为推荐结尾
    
    评价格式如下：
    评分：[X] 星（满分 5 星）
    
    优点：
    1. [优点 1]
    2. [优点 2]
    ...
    
    缺点：
    1. [缺点 1]
    2. [缺点 2]
    ...
    
    推荐：[一句话推荐]
    """,
)

In [22]:
input_variables = {
    "product": "智能手机 X",
    "rating": "4",
    "pros": "3",
    "cons": "2",
    "word_limit": "200",
}

chain = review_prompt | llm
output = chain.invoke(input_variables).content
display(Markdown(output))

评分：4 星（满分 5 星）

优点：  
1. 智能手机 X 的屏幕显示效果非常出色，色彩鲜艳且细节清晰，非常适合观看视频和浏览图片。即使在阳光下，屏幕亮度也足够高，确保内容依然可见。  
2. 电池续航表现令人满意，正常使用情况下可以轻松坚持一整天。配合快速充电功能，短时间内即可恢复大量电量，减少等待时间。  
3. 相机拍摄能力强大，尤其是在低光环境下表现出色。多种拍摄模式和AI优化功能让照片更加生动，满足日常拍摄需求。  

缺点：  
1. 手机的机身设计略显厚重，长时间单手握持会感到疲劳。与同类产品相比，便携性稍逊一筹。  
2. 系统界面虽然流畅，但预装软件较多，部分应用无法卸载，占用了一定的存储空间。这可能会让喜欢简洁系统的用户感到不便。  

推荐：智能手机 X 是一款性能均衡、拍照出色的设备，适合追求高性价比的用户。